<a href="https://colab.research.google.com/github/KimMunho/ComputerVision/blob/main/HW4/HW4_Sterp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =======================================
# 0) Google Drive 마운트 & YOLOv5 경로 설정
# =======================================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

repo_path = '/content/drive/MyDrive/ColabNotebooks/ComputerVision/HW4/yolov5'  # YOLOv5 root
%cd {repo_path}

import sys, os, time, glob, torch, pandas as pd, shutil
sys.path.append(repo_path)        # 모듈 탐색 경로 등록

# 의존성 설치(이미 설치돼 있다면 1-2초 만에 끝남)
!pip install -qr requirements.txt
print('✅ requirements install done')

# =======================================
# 1) COCO128 데이터셋 준비
# =======================================
if not os.path.isdir('coco128'):
    !curl -L "https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip" -o coco128.zip -s
    !unzip -q coco128.zip
print('✅ COCO128 ready')

# =======================================
# 2) 학습
# =======================================
!python train.py --img 640 --batch 16 --epochs 50 --data coco128.yaml \
                 --weights yolov5s.pt --project {repo_path}/runs/train \
                 --name my_yolo_exp --exist-ok
print('✅ training finished')

# =======================================
# 3) 추론 속도 벤치마크
# =======================================
from utils.torch_utils import select_device
from models.common import DetectMultiBackend
from utils.dataloaders import LoadImages   # ← 변경된 위치

weights = f'{repo_path}/runs/train/my_yolo_exp/weights/best.pt'
device  = select_device('')
model   = DetectMultiBackend(weights, device=device)

loader  = LoadImages('data/images', img_size=640, stride=model.stride, auto=True)
_ = [model(img.to(device)) for _, img, *_ in zip(range(3), loader)]  # warm-up
N, t0 = 0, time.perf_counter()
for _, img, *_ in loader:
    _ = model(img.to(device)); N += 1
fps = N / (time.perf_counter() - t0)
print(f"🚀 {fps:.2f} FPS on {device.type.upper()} (640×640)")

# =======================================
# 4) 성능 지표 요약 표
# =======================================
df = pd.read_csv(f'{repo_path}/runs/train/my_yolo_exp/results.csv')
from tabulate import tabulate
row = ["YOLOv5s",
       f"{df['metrics/mAP_0.5'].iloc[-1]:.3f}",
       f"{df['metrics/precision'].iloc[-1]:.3f}",
       f"{df['metrics/recall'].iloc[-1]:.3f}",
       f"{fps:.2f}"]
print(tabulate([row], headers=["Model","mAP@0.5","Precision","Recall","FPS"], tablefmt="github"))

# =======================================
# 5) 그래프 PNG를 HW4 폴더 루트에 복사
# =======================================
for png in ['results.png', 'PR_curve.png', 'confusion_matrix.png']:
    src = f'{repo_path}/runs/train/my_yolo_exp/{png}'
    dst = f'{repo_path}/{png}'
    if os.path.exists(src):
        shutil.copy(src, dst)
print('✅ PNG files copied to HW4 root')

# =======================================
# 6) 간단 검출 데모
# =======================================
!python detect.py --weights {repo_path}/runs/train/my_yolo_exp/weights/best.pt \
                  --img 640 --source data/images --conf 0.25 --save-txt --save-conf --exist-ok
print('🏁 all done')


Mounted at /content/drive
/content/drive/MyDrive/ColabNotebooks/ComputerVision/HW4/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.4 MB/s eta 0:00:00
✅ requirements install done
✅ COCO

YOLOv5 🚀 v7.0-419-gcd44191c Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
